<a href="https://colab.research.google.com/github/rezlevi/Big-Project/blob/main/testprogram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
from sklearn.metrics import mean_squared_error
from statistics import mean
import random
import tensorflow as tf
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
HIST = 10
COLS = 4
adatok_nyers=[sor.strip().split(sep=";") for sor in open("/content/drive/My Drive/BP_d.csv.txt")]
adatok = []
for i in range(1,len(adatok_nyers)):
  adatok.append([float(adatok_nyers[i][1]), float(adatok_nyers[i][2]), float(adatok_nyers[i][3]), float(float(adatok_nyers[i][2]) - float(adatok_nyers[i][3])) ])

In [ ]:
train=adatok[:20000]
test=adatok[20000:25000]

In [ ]:
import numpy as np
train_x=(np.array([ train[i:i+HIST] for i in range(len(train)-HIST)])-8)/40
test_x=(np.array([ test[i:i+HIST] for i in range(len(test)-HIST)])-8)/40
train_y=(np.array([ train[i+HIST] for i in range(len(train)-HIST)])-8)/40
test_y=(np.array([ test[i+HIST] for i in range(len(test)-HIST)])-8)/40


print(train_x[0])
print(train_y[0])

[[-0.3425 -0.21   -0.43    0.02  ]
 [-0.4325 -0.365  -0.4825 -0.0825]
 [-0.4275 -0.365  -0.47   -0.095 ]
 [-0.475  -0.445  -0.51   -0.135 ]
 [-0.4775 -0.425  -0.5875 -0.0375]
 [-0.4    -0.35   -0.545  -0.005 ]
 [-0.3375 -0.275  -0.3725 -0.1025]
 [-0.45   -0.3475 -0.51   -0.0375]
 [-0.5025 -0.425  -0.5975 -0.0275]
 [-0.505  -0.445  -0.6225 -0.0225]]
[-0.4325 -0.39   -0.5375 -0.0525]


In [ ]:
functions = ['tanh', 'sigmoid', 'exponential', 'hard_sigmoid', 'linear', 'relu']

In [ ]:
def train1(func): ##with the same function
  print('train with tensorflow')
  model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(HIST,COLS)),
    tf.keras.layers.Dense(100, activation= func),
    tf.keras.layers.Dense(50, activation= func),
    tf.keras.layers.Dense(COLS, activation= func)
  ])

  model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

  mselst=[]
  for _ in range(50):
    model.fit(train_x, train_y, epochs=1)
    y=model.predict(test_x)
    print('prediction MSE:', mean_squared_error(test_y, y))
    print('dummy MSE:', mean_squared_error(test_y[HIST:], test_y[HIST-1:-1]))
    mselst.append(mean_squared_error(test_y, y))
  return mean(mselst)

In [ ]:
def train2(func1,func2,func3): ##with different functions
  print('train with tensorflow')
  model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(HIST,COLS)),
    tf.keras.layers.Dense(100, activation= func1),
    tf.keras.layers.Dense(50, activation= func2),
    tf.keras.layers.Dense(COLS, activation= func3)
  ])

  model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

  mselst=[]
  for _ in range(50):
    model.fit(train_x, train_y, epochs=1)
    y=model.predict(test_x)
    print('prediction MSE:', mean_squared_error(test_y, y))
    print('dummy MSE:', mean_squared_error(test_y[HIST:], test_y[HIST-1:-1]))
    mselst.append(mean_squared_error(test_y, y))
  return mean(mselst)

In [ ]:
def train3(x,y): 
  print('train with tensorflow')
  model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(HIST,COLS)),
    tf.keras.layers.Dense(x, activation= 'tanh'),
    tf.keras.layers.Dense(y, activation= 'relu'),
    tf.keras.layers.Dense(COLS, activation= 'tanh')
  ])

  model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

  mselst=[]
  for _ in range(50):
    model.fit(train_x, train_y, epochs=1)
    y=model.predict(test_x)
    print('prediction MSE:', mean_squared_error(test_y, y))
    print('dummy MSE:', mean_squared_error(test_y[HIST:], test_y[HIST-1:-1]))
    mselst.append(mean_squared_error(test_y, y))
  return mean(mselst)

In [ ]:
best_result = 100000000 ##tanh
best_function = ''
result = 0
for i in range(len(functions)):
  func = functions[i]
  result = train1(func)
  if result < best_result:
    best_result = result
    best_function = func

print(best_result)
print(best_function)

In [ ]:
best_result = 100000000 ##tanh, relu, tanh
best_functions = []
result = 0
for i in range(10):
  func1 = functions[random.randint(0,5)]
  func2 = functions[random.randint(0,5)]
  func3 = functions[random.randint(0,5)]
  result = train2(func1,func2,func3)
  if result < best_result:
    best_result = result
    best_functions = [func1, func2,func3]

print(best_result)
print(best_functions)

In [ ]:
#tanh tanh tanh vs tanh relu tanh: B B B
result1 = train1('tanh')
result2 = train2('tanh','relu','tanh')
if(result1 < result2):
  print('A')
else:
  print('B')

In [ ]:
best_result = 100000000 ## Number testing
best_numbers = []
result = 0
for i in range(10):
  x = random.randint(50,150)
  y = random.randint(0,50)
  if i == 0:
    x = 80
    y= 32
  if i == 1:
    x=124
    y=23
  result = train3(x,y)
  if result < best_result:
    best_result = result
    best_numbers = [x,y]

print(best_result)
print(best_numbers) ## 100,50  -> 80,32 -> 124,23 -> 111,42

train with tensorflow
156/156 [==============================] - 0s 1ms/step
prediction MSE: 0.0037556439308789906
dummy MSE: 0.004371728162650618
156/156 [==============================] - 0s 1ms/step
prediction MSE: 0.0037051806479562457
dummy MSE: 0.004371728162650618
156/156 [==============================] - 0s 1ms/step
prediction MSE: 0.0036373982232684463
dummy MSE: 0.004371728162650618
156/156 [==============================] - 0s 1ms/step
prediction MSE: 0.003631546844252802
dummy MSE: 0.004371728162650618
156/156 [==============================] - 0s 1ms/step
prediction MSE: 0.00359822583115256
dummy MSE: 0.004371728162650618
156/156 [==============================] - 0s 2ms/step
prediction MSE: 0.0035944719235154405
dummy MSE: 0.004371728162650618
156/156 [==============================] - 0s 2ms/step
prediction MSE: 0.003640849782517581
dummy MSE: 0.004371728162650618
156/156 [==============================] - 0s 1ms/step
prediction MSE: 0.0034914770663280596
dummy MSE: 0.0